In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm_notebook
from sklearn.metrics import accuracy_score, f1_score
import torchvision
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
classes = {}
pt = 0
for i in pd.read_csv('./cifar-10/trainLabels.csv')['label'].unique():
    ar = [0,0,0,0,0,0,0,0,0,0]
    ar[pt] = 1
    pt+=1
    classes[i] = ar

In [3]:
def get_onehot(label):
    return classes[label]

In [4]:
class SupervisedDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, df, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = df
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                str(self.df.iloc[idx, 0])+'.png')
        image = Image.open(img_name)
        label = self.df.iloc[idx, 1]
        onehot = np.array(get_onehot(label))
#         landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'label': torch.Tensor(onehot).to('cuda')}

        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample

In [5]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 5 * 5)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

model =  torchvision.models.resnext50_32x4d(pretrained=True, progress=True).cuda()

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /home/nikamanth/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


In [6]:
df = pd.read_csv('./cifar-10/trainLabels.csv')
df = df.sample(frac=1)
traindataset = SupervisedDataset(df[:int(len(df)*0.75)],root_dir='./cifar-10/train/',
                                        transform=
                                        transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
trainloader = DataLoader(traindataset, batch_size=2,shuffle=True)

testdataset = SupervisedDataset(df[int(len(df)*0.75):],root_dir='./cifar-10/train/',
                                        transform=
                                        transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
testloader = DataLoader(testdataset, batch_size=2, shuffle=True)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [8]:
train_f1 = []
dev_f1 = []
num_of_epochs=30
device = 'cuda'
y_pred = []
y_true = []
for epoch in range(num_of_epochs):
    i_batch = 0
    print("epoch:",epoch)
    for batch in tqdm_notebook(trainloader):
        i_batch +=1
#         print(batch['image'].shape)
#         break
#     break
        output = model(batch['image'].float().to(device))
        loss = criterion(output,batch['label'].argmax(dim=1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_pred += list(output.argmax(dim=1).detach().cpu().numpy())
        y_true += list(batch['label'].argmax(dim=1).detach().cpu().numpy()) 
    print("training acc:",accuracy_score(y_pred,y_true),end=' ')
    f1 = f1_score(y_pred,y_true,average='macro')
    train_f1.append(f1)
    print("training f1_score:", f1)
    
    y_pred = []
    y_true = []
    for batch in testloader:
        output = model(batch['image'].float().to(device))
        y_pred += list(output.argmax(dim=1).detach().cpu().numpy())
        y_true += list(batch['label'].argmax(dim=1).detach().cpu().numpy())
    print("dev acc:",accuracy_score(y_pred,y_true),end=' ')
    f1 = f1_score(y_pred,y_true,average='macro')
    dev_f1.append(f1)
    print("training f1_score:", f1)

epoch: 0


KeyboardInterrupt: 